In [2]:
!pip install pycocotools

  Obtaining dependency information for pycocotools from https://files.pythonhosted.org/packages/ba/64/0451cf41a00fd5ac4501de4ea0e395b7d909e09d665e56890b5d3809ae26/pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 9.2 MB/s eta 0:00:00:00:01


In [3]:
!rm -rf /kaggle/working/*

In [4]:
!git clone https://github.com/dang22mse23054/mse11_capstone
!ls -la

Cloning into 'mse11_capstone'...
remote: Enumerating objects: 425, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 425 (delta 22), reused 25 (delta 12), pack-reused 384
Receiving objects: 100% (425/425), 17.99 MiB | 12.37 MiB/s, done.
Resolving deltas: 100% (276/276), done.
total 16
drwxr-xr-x 4 root root 4096 Jan  7 02:22 .
drwxr-xr-x 5 root root 4096 Jan  7 02:20 ..
drwxr-xr-x 2 root root 4096 Jan  7 02:20 .virtual_documents
drwxr-xr-x 6 root root 4096 Jan  7 02:22 mse11_capstone


In [5]:
!echo "cd mse11_capstone" > git_fetch.sh
!echo "git fetch" >> git_fetch.sh
!echo "git checkout kaggle" >> git_fetch.sh
!echo "git pull" >> git_fetch.sh
!cat git_fetch.sh

cd mse11_capstone
git fetch
git checkout kaggle
git pull


In [6]:
!pwd
!bash git_fetch.sh
!cp /kaggle/working/mse11_capstone/source_code/age_gender/* ./
!cp -r /kaggle/working/mse11_capstone/source_code/models ./
!cp -r /kaggle/working/mse11_capstone/source_code/common ./
!ls

/kaggle/working
Branch 'kaggle' set up to track remote branch 'kaggle' from 'origin'.
Switched to a new branch 'kaggle'
Already up to date.
common	       dataset.py    model.py  mse11_capstone  test.py
dataloader.py  git_fetch.sh  models    predict.py      trainer.py


In [ ]:
from dataloader import AgeGenderDataLoader
from model import AgeGenderDetectionModel
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

if __name__ == "__main__":
    model = AgeGenderDetectionModel(
        lr = 1e-3,
        momentum = 0.9,
        weight_decay = 1e-4,        
    )

    data = AgeGenderDataLoader(batch_size=64, workers=4)

    trainer = Trainer(
        accelerator="gpu",
        # accelerator="mps",
        # checkpoint_callback=True,
        callbacks = [
            LearningRateMonitor(logging_interval='step'),
            ModelCheckpoint(filename='{epoch}-{val_acc:.4f}-{val_age_acc:.4f}-{val_gender_acc:.4f}', save_top_k=2, monitor='val_acc', mode='max'),
        ], 
        # check_val_every_n_epoch=1,
        fast_dev_run=False,
        default_root_dir='checkpoint',
        max_epochs=25,       
    )

    trainer.fit(model, data)

In [7]:
!zip -r file.zip /kaggle/working/checkpoint

  adding: kaggle/working/checkpoint/ (stored 0%)
  adding: kaggle/working/checkpoint/lightning_logs/ (stored 0%)
  adding: kaggle/working/checkpoint/lightning_logs/version_0/ (stored 0%)
  adding: kaggle/working/checkpoint/lightning_logs/version_0/events.out.tfevents.1704554336.44df265b5b52.42.0 (deflated 74%)
  adding: kaggle/working/checkpoint/lightning_logs/version_0/checkpoints/ (stored 0%)
  adding: kaggle/working/checkpoint/lightning_logs/version_0/checkpoints/epoch=20-val_acc=0.8411-val_age_acc=0.7983-val_gender_acc=0.8886.ckpt (deflated 10%)
  adding: kaggle/working/checkpoint/lightning_logs/version_0/checkpoints/epoch=24-val_acc=0.8411-val_age_acc=0.8013-val_gender_acc=0.8857.ckpt (deflated 10%)
  adding: kaggle/working/checkpoint/lightning_logs/version_0/hparams.yaml (deflated 16%)


In [8]:
!ls -la

total 518180
drwxr-xr-x 8 root root      4096 Jan  6 16:01 .
drwxr-xr-x 5 root root      4096 Jan  6 15:17 ..
drwxr-xr-x 2 root root      4096 Jan  6 15:17 .virtual_documents
drwxr-xr-x 2 root root      4096 Jan  6 15:18 __pycache__
drwxr-xr-x 3 root root      4096 Jan  6 15:18 checkpoint
drwxr-xr-x 3 root root      4096 Jan  6 15:18 common
-rw-r--r-- 1 root root      3444 Jan  6 15:18 dataloader.py
-rw-r--r-- 1 root root      4069 Jan  6 15:18 dataset.py
-rw-r--r-- 1 root root 530553058 Jan  6 16:01 file.zip
-rw-r--r-- 1 root root        59 Jan  6 15:18 git_fetch.sh
-rw-r--r-- 1 root root      6643 Jan  6 15:18 model.py
drwxr-xr-x 3 root root      4096 Jan  6 15:18 models
drwxr-xr-x 6 root root      4096 Jan  6 15:18 mse11_capstone
-rw-r--r-- 1 root root      3555 Jan  6 15:18 predict.py
-rw-r--r-- 1 root root       996 Jan  6 15:18 trainer.py


In [9]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip

In [8]:
import torch
from dataloader import AgeGenderDataLoader
from model import AgeGenderDetectionModel
from pytorch_lightning import Trainer

if __name__ == "__main__":
    ckp_file = '/kaggle/input/age-gender-model/epoch24-val_acc0.8411-val_age_acc0.8013-val_gender_acc0.8857.ckpt'
    checkpoint = torch.load(ckp_file, map_location=torch.device('cpu'))
    model = AgeGenderDetectionModel()
    model.load_state_dict(checkpoint['state_dict'])

    data = AgeGenderDataLoader(batch_size=64, workers=4)

    trainer = Trainer(
        # accelerator="cpu",
        accelerator="gpu",
    )

    trainer.test(model, data)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(Mode test) Total: 23708, Train: 18968, Validation: 2370, Test: 2370
(Mode test) Correct Partition
                                        List
0      80_0_0_20170117175258116.jpg.chip.jpg
1      15_0_0_20170110232331351.jpg.chip.jpg
2      26_1_4_20170117202038959.jpg.chip.jpg
3       1_1_2_20161219140803696.jpg.chip.jpg
4      22_0_0_20170104230045815.jpg.chip.jpg
...                                      ...
18963  30_0_2_20170116173516742.jpg.chip.jpg
18964   1_1_2_20161219193510436.jpg.chip.jpg
18965  27_1_2_20170116190003226.jpg.chip.jpg
18966  67_0_1_20170120223307141.jpg.chip.jpg
18967  80_0_2_20170111210646563.jpg.chip.jpg

[18968 rows x 1 columns]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8609620511331141     │
│       test_age_acc        │    0.8143715858459473     │
│      test_gender_acc      │     0.907317578792572     │
└───────────────────────────┴───────────────────────────┘